<a href="https://colab.research.google.com/github/BenKohn2004/Fencing_Tracking_Example/blob/main/Detection_and_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Main Code for Training the Model

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="8rRReZyvWqputItwBIl0")
project = rf.workspace("benkohn200412").project("bellguard")
dataset = project.version(5).download("yolov8")


In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True

Main Code for Detection and Tracking

In [ ]:
!pip install ultralytics
!pip install roboflow --quiet

from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import csv
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path to the model weights
model_weights = '/content/drive/MyDrive/weights/best.pt'

# Set the path to the video file
video_path = '/content/drive/MyDrive/output_videos/output_691920.mp4'

# Define the paths and filenames
output_csv_path = "/content/drive/MyDrive/positions.csv"  # Output CSV file path in Google Drive

Runs all videos in Output_Videos

In [ ]:
import os
import random

# Path to the directory containing the video files
video_directory = "/content/drive/MyDrive/output_videos"

# Get a list of video files in the directory
video_files = os.listdir(video_directory)

# # Iterate over all video files
# for video_file in video_files:
#     video_path = os.path.join(video_directory, video_file)
#     result_video_file = "output_" + video_file
#     result_video_path = os.path.join(video_directory, result_video_file)

#     if not os.path.exists(result_video_path):
#         acceleration = run_inference(video_path)

# Iterate over all video files
# for video_file in video_files:
video_file = "output_775560.mp4"
video_path = os.path.join(video_directory, video_file)
result_video_file = "output_" + video_file
result_video_path = os.path.join(video_directory, result_video_file)

if not os.path.exists(result_video_path):
    acceleration = run_inference(video_path)

Runs Just the Videos in a List

In [40]:
import os

# Path to the directory containing the video files
video_directory = "/content/drive/MyDrive"

# List of video files to process
video_files = ["Clip 1.mp4", "Clip 2.mp4", "Clip 3.mp4"]

# Iterate over video files
for video_file in video_files:
    video_path = os.path.join(video_directory, video_file)
    result_video_file = "output_" + video_file
    result_video_path = os.path.join(video_directory, result_video_file)

    if not os.path.exists(result_video_path):
        acceleration = run_inference(video_path)


<class 'str'>
Clip 1.mp4
Running Clips 1, 2 or 3.
[[455.5, 1385.0, 0, 0], [453.5, 1385.5, 1, 0], [453.0, 1385.5, 1, 0], [451.5, 1385.5, 1, 0], [450.0, 1385.0, 1, 0], [448.5, 1385.5, 1, 0], [447.5, 1385.5, 1, 0], [446.0, 1385.5, 1, 0], [446.0, 1385.5, 1, 0], [445.0, 1385.5, 1, 0], [445.0, 1384.5, 1, 0], [444.0, 1384.5, 1, 0], [444.0, 1384.5, 1, 0], [444.0, 1384.5, 1, 0], [444.0, 1384.5, 1, 0], [444.0, 1384.5, 1, 0], [444.0, 1384.0, 1, 0], [444.5, 1384.0, 1, 0], [444.5, 1384.0, 1, 0], [444.5, 1384.0, 1, 0], [444.5, 1384.0, 1, 0], [444.5, 1384.0, 1, 0], [445.0, 1384.0, 1, 0], [446.5, 1383.5, 1, 0], [449.0, 1381.0, 1, 0], [452.0, 1377.5, 1, 0], [453.0, 1376.5, 1, 0], [455.5, 1373.0, 1, 0], [460.5, 1370.0, 1, 0], [470.5, 1366.5, 1, 0], [481.0, 1364.0, 1, 0], [496.5, 1360.0, 1, 0], [502.5, 1358.5, 1, 0], [520.0, 1353.0, 1, 0], [548.1, 1343.0, 1, 0], [576.2, 1327.0, 1, 0], [604.3000000000001, 1309.5, 1, 0], [632.4000000000001, 1296.0, 1, 0], [660.5, 1282.5, 1, 0], [685.5833333333334, 1269.5, 

In [39]:
def run_inference(video_path):
  # Initialize the YOLO model
  model = YOLO(model_weights)

  cap = cv2.VideoCapture(video_path)
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  # Get the name of the input video file
  video_filename = os.path.basename(video_path)
  video_name = os.path.splitext(video_filename)[0]

  # Define the output video file path on Google Drive
  output_drive_path = "/content/drive/MyDrive/Tracked_Videos/output_{}.mp4".format(video_name)

  # Define the codec and create VideoWriter object
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_drive_path, fourcc, 30.0, (frame_width, frame_height))

  frame_count = 1
  fencer_initialization_frames = 5

  # Maintains a List of the all the Fencer Values for a given clip
  left_fencer_list = []
  right_fencer_list = []
  # Maintains a List of which bellguard locations are detected or estimated, 0 is detected, 1 is estimated
  left_certainty_list = []
  right_certainty_list = []

  while(True):
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, conf = 0.50, persist=True, verbose=False)

    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    classes = results[0].boxes.cls.cpu().numpy().astype(int)
    confidences = results[0].boxes.conf.cpu().numpy()
    ids = results[0].boxes.id.cpu().numpy().astype(int)

    left_fencer, right_fencer = Fencer(), Fencer()

    if frame_count <= fencer_initialization_frames:
      # Creates the initial conditions for the fencers
      left_fencer, right_fencer = create_fencers(left_fencer, right_fencer, boxes, classes, confidences, frame_width, frame_height)
      left_certainty, right_certainty = 0, 0
    else:
      frame, left_fencer, left_certainty = track_fencers(frame, boxes, classes, confidences, left_fencer, left_fencer_list[-1], left_fencer_list[-2], "Left", frame_width, frame_height, left_certainty_list[-1], left_certainty_list[-2])
      frame, right_fencer, right_certainty = track_fencers(frame, boxes, classes, confidences, right_fencer, right_fencer_list[-1], right_fencer_list[-2], "Right", frame_width, frame_height, right_certainty_list[-1], right_certainty_list[-2])

    # Draws the bounding boxes
    for box, id in zip(boxes, ids):
      cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
      cv2.putText(
          frame,
          f"Id {id}",
          (box[0], box[1]),
          cv2.FONT_HERSHEY_SIMPLEX,
          1,
          (0, 0, 255),
          2,
      )

    # Performs a Linear Interpolation on the Bellguard Position
    left_fencer_list  = linear_interpolation(left_fencer_list, left_certainty_list)
    right_fencer_list = linear_interpolation(right_fencer_list, right_certainty_list)

    # Draw the Left bellguard
    cv2.circle(frame, (int(left_fencer.bellguard_x),int(left_fencer.bellguard_y)), 5, (0, 0, 255), -1)
    # Draw the Right bellguard
    cv2.circle(frame, (int(right_fencer.bellguard_x),int(right_fencer.bellguard_y)), 5, (0, 255, 0), -1)

    # Draw the Left torso
    cv2.circle(frame, (int(left_fencer.torso_x),int(left_fencer.torso_y)), 5, (255, 0, 255), -1)
    # Draw the Right torso
    cv2.circle(frame, (int(right_fencer.torso_x),int(right_fencer.torso_y)), 5, (255, 255, 0), -1)

    # Builds the fencer lists
    left_fencer_list.append(left_fencer)
    left_certainty_list.append(left_certainty)
    right_fencer_list.append(right_fencer)
    right_certainty_list.append(right_certainty)

    # Process the frame and write it to the video writer
    out.write(frame)

    frame_count += 1

  # Release the video writer
  out.release()

  # # Trim video file name to get the desired value
  trimmed_video_name = video_file.replace("output_", "").replace(".mp4", "")

  print(type(video_path[-10:]))
  print(video_path[-10:])

  if video_path[-10:] == "Clip 1.mp4" or video_path[-10:] == "Clip 2.mp4" or video_path[-10:] == "Clip 3.mp4":
    print("Running Clips 1, 2 or 3.")
    if video_path[-10:] == "Clip 1.mp4":
      red_light = ""
      green_light = 51
    elif video_path[-10:] == "Clip 2.mp4":
      green_light = 73
      red_light = 77
    elif video_path[-10:] == "Clip 3.mp4":
      green_light = 79
      red_light = 80
  else:

    # Open the "clips.csv" file
    csv_file_path = "/content/drive/MyDrive/clips.csv"
    with open(csv_file_path, "r") as file:
        reader = csv.DictReader(file)

        # Search for the row with matching Middle_Frame value
        for row in reader:
            if row["Middle_Frame"] == trimmed_video_name:
                red_light = row["Red_Light"]
                green_light = row["Green_Light"]
                break

    # Print the retrieved values
    print(video_file)
    print("Red Light:", red_light)
    print("Green Light:", green_light)

  # Create a list of fencer positions
  pos_list = []
  for i in range(len(left_fencer_list)):
    a = []
    a.append(left_fencer_list[i].bellguard_x)
    a.append(right_fencer_list[i].bellguard_x)
    # In case of one light, the red or green light may not exist.
    try:
      red_light = int(float(red_light))
      if i + 1 < red_light:
        a.append(0)
      else:
        a.append(1)
    except:
      a.append(0)
      red_light = 0
    try:
      green_light = int(float(green_light))
      if i + 1 < green_light:
        a.append(0)
      else:
        a.append(1)
    except:
      a.append(0)
      green_light = 0
    pos_list.append(a)

  print(pos_list)

  # Creates a List of the Fencer Velocities
  velocity = []
  for i in range(len(pos_list) - 1):
    velocity.append([pos_list[i+1][0]-pos_list[i][0], pos_list[i+1][1]-pos_list[i][1], pos_list[i][2], pos_list[i][3]])
  print(velocity)
  acceleration = []

  # Only runs the acceleration until a light turns on
  if red_light is None and green_light is None:
    maximum = None
  elif red_light is None:
    maximum = green_light
  elif green_light is None:
    maximum = red_light
  else:
    maximum = max(red_light, green_light)

  print(f'The maximum number of frames analyzed for acceleration is {maximum}. Red_Light is {red_light} and Green_Light is {green_light}.')

  # The Right Fencer's Acceleration is flipped so that both accelerations are measured with respect to the opponent
  for i in range(maximum):
    acceleration.append([velocity[i+1][0]-velocity[i][0], velocity[i][1]-velocity[i+1][1], pos_list[i][2], pos_list[i][3]])
  acceleration = [[max(frame_width/100*-1, min(frame_width/100, value)) for value in sublist] for sublist in acceleration]
  acceleration = [[subset[0] / (frame_width/100), subset[1] / (frame_width/100)] + subset[2:] for subset in acceleration]
  print(acceleration)

  # Saves the Acceleration File
  # Define the output file path with the video file name
  output_file_name = f"acceleration_{video_name}.csv"
  output_file_path = os.path.join("/content/drive/MyDrive/Acceleration_Files/", output_file_name)

  # Define the column headers
  headers = ["Left", "Right", "Red_Light", "Green_Light"]

  # Save the acceleration list to the output file
  with open(output_file_path, mode='w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(headers)  # Write the column headers
      writer.writerows(acceleration)

  print("Acceleration list saved:", output_file_path)


  return acceleration

In [3]:
def linear_interpolation(fencer_list, certainty_list):
  # Sliding Window to perform linear interpolation of fencer bellguard position
  # 0 is a certain point, 1 is an uncertain point

  if len(certainty_list) < 3:
    return fencer_list

  l, r= 0 , 1

  while r < len(certainty_list):
    # Increments l with r until a 1 is reached from a 0
    if certainty_list[r] == 1 and certainty_list[r-1] == 0:
      l = r
    # If r is at a 0 that is reached from a 1 then perform interpolation
    elif certainty_list[r] == 0 and certainty_list[r-1] == 1 and l != 0:
      # Finds the difference in position of two certain points, divided by the number of steps
      delta_x = (fencer_list[r].bellguard_x - fencer_list[l - 1].bellguard_x)/(r-l+1)
      for j in range(r-l):
        fencer_list[l+j].bellguard_x = fencer_list[l+j-1].bellguard_x + delta_x
      # Moves l up to r
      l = r
    r += 1

  return fencer_list

In [4]:
class Fencer:
    def __init__(self, torso_x=0, torso_y=0, torso_height=0, torso_width=0, bellguard_x=0, bellguard_y=0, torso_frame=0, bell_frame = 0):
        self.torso_x = torso_x
        self.torso_y = torso_y
        self.torso_height = torso_height
        self.torso_width = torso_width
        self.bellguard_x = bellguard_x
        self.bellguard_y = bellguard_y
        # Frames for torso and bell help with a running average and allow for a missed detection in the initialization
        self.torso_frame = torso_frame
        self.bell_frame = bell_frame

In [5]:
def track_fencers(frame, boxes, classes, confidences, fencer, prev_fencer, prev_prev_fencer, side, frame_width, frame_height, prev_certainty, prev_prev_certainty):

# Torso Tracking

  # Define the Speed of the Bellguard, works for both Left and Right
  x_speed = prev_fencer.torso_x - prev_prev_fencer.torso_x
  y_speed = prev_fencer.torso_y - prev_prev_fencer.torso_y

  # The Expected Bellguard position, works for both Left and Right
  expected_x = prev_fencer.torso_x + x_speed
  expected_y = prev_fencer.torso_y + y_speed

  x1, x2 = prev_fencer.torso_x - prev_fencer.torso_width/2, prev_fencer.torso_x + prev_fencer.torso_width/2
  y1, y2 = prev_fencer.torso_y - prev_fencer.torso_height/2, prev_fencer.torso_y + prev_fencer.torso_height/2

  # Cycle through the Detections and
  for i in range(len(boxes)):
    # Finds the center of the detection
    x_det = (boxes[i][2] + boxes[i][0])/2
    y_det = (boxes[i][3] + boxes[i][1])/2
    # Must be identified as a Torso and greater than a tenth of the image height
    if classes[i] == 3 and boxes[i][3] - boxes[i][1] > frame_height/10:
      # Checks if the detection is in the detection box and has not yet been assigned
      if x1 < x_det and x_det < x2 and y1 < y_det and y_det < y2 and fencer.torso_x == 0:
        fencer.torso_x, fencer.torso_y = x_det, y_det

  if fencer.torso_x == 0:
    fencer.torso_x, fencer.torso_y = expected_x, expected_y

  # Fencer Torso Height and Width are assumed not to change from frame to frame
  fencer.torso_height, fencer.torso_width = prev_fencer.torso_height, prev_fencer.torso_width

# Bellguard Tracking

  # Define the Speed of the Bellguard, works for both Left and Right
  x_speed = prev_fencer.bellguard_x - prev_prev_fencer.bellguard_x
  y_speed = prev_fencer.bellguard_y - prev_prev_fencer.bellguard_y

  # The Expected Bellguard position, works for both Left and Right
  expected_x = prev_fencer.bellguard_x + x_speed
  expected_y = prev_fencer.bellguard_y + y_speed

  # Define the Tracking Box for allowable detections
  # The bellguard should not be behind the torso center
  if side == "Left":
    x1 = prev_fencer.torso_x
  else:
    x2 = prev_fencer.torso_x
  # The bellguard should not be more than a fencer's torso away from the expected position
  # The Certainty allows for a larger search box if the position was not previously found
  if side == "Left":
    x2 = expected_x + 1*prev_fencer.torso_width*(1+prev_certainty+prev_prev_certainty)
  else:
    x1 = expected_x - 1*prev_fencer.torso_width*(1+prev_certainty+prev_prev_certainty)
  # The bellguard should not be above the fencers head, which is included in the torso detection
  y1 = prev_fencer.torso_y - prev_fencer.torso_height/4
  # The bellguard should not be below half of the torso height below the bottom of the torso, about knee level
  y2 = prev_fencer.torso_y + prev_fencer.torso_height

  # Cycle through the Detections and
  for i in range(len(boxes)):
    # Finds the center of the detection
    x_det = (boxes[i][2] + boxes[i][0])/2
    y_det = (boxes[i][3] + boxes[i][1])/2
    if classes[i] == 0:
      # Checks if the detection is in the detection box and has not yet been assigned
      if x1 < x_det and x_det < x2 and y1 < y_det and y_det < y2 and fencer.bellguard_x == 0:
        fencer.bellguard_x, fencer.bellguard_y = x_det, y_det
        # Assigns a certainty of 0
        certainty = 0

  # If fencer bellguard x is still 0 after cycling through the detections, then the expected position is assigned to the bellguard
  if fencer.bellguard_x == 0:
    # Checks if the previous two detections have been missed
    if prev_certainty + prev_certainty < 2:
      if side == "Left":
        fencer.bellguard_x, fencer.bellguard_y = max(fencer.torso_x+fencer.torso_width/2,expected_x), expected_y
      else:
        fencer.bellguard_x, fencer.bellguard_y = min(fencer.torso_x-fencer.torso_width/2,expected_x), expected_y
    # Assumes a position if the two previous detections have been missed
    else:
      if side == "Left":
        fencer.bellguard_x, fencer.bellguard_y = fencer.torso_x+fencer.torso_width*1.25, expected_y
      else:
        fencer.bellguard_x, fencer.bellguard_y = fencer.torso_x-fencer.torso_width*1.25, expected_y

    # Assigns a certainty of 1
    certainty = 1

  return frame, fencer, certainty

In [6]:
def create_fencers(left_fencer, right_fencer, boxes, classes, confidences, frame_width, frame_height):

  # Allows only two detections for Torso and Bellguard
  box_num = 0
  bell_num = 0

  for i in range(len(boxes)):
    if classes[i] == 3 and box_num < 2 and boxes[i][3] - boxes[i][1] > frame_height/10:
      # Left Fencer
      if boxes[i][0] < frame_width/2:
        # Provides a Running Average
        left_fencer.torso_x = (left_fencer.torso_x*left_fencer.torso_frame + (boxes[i][0] + boxes[i][2])/2)/(left_fencer.torso_frame + 1)
        left_fencer.torso_y = (left_fencer.torso_y*left_fencer.torso_frame + (boxes[i][1] + boxes[i][3])/2)/(left_fencer.torso_frame + 1)
        left_fencer.torso_width = (left_fencer.torso_width*left_fencer.torso_frame + (boxes[i][2] - boxes[i][0]))/(left_fencer.torso_frame + 1)
        left_fencer.torso_height = (left_fencer.torso_height*left_fencer.torso_frame + (boxes[i][3] - boxes[i][1]))/(left_fencer.torso_frame + 1)
        left_fencer.torso_frame += 1
      else:
        right_fencer.torso_x = (right_fencer.torso_x*right_fencer.torso_frame + (boxes[i][0] + boxes[i][0])/2)/(right_fencer.torso_frame + 1)
        right_fencer.torso_y = (right_fencer.torso_y*right_fencer.torso_frame + (boxes[i][1] + boxes[i][3])/2)/(right_fencer.torso_frame + 1)
        right_fencer.torso_width = (right_fencer.torso_width*right_fencer.torso_frame + (boxes[i][2] - boxes[i][0]))/(right_fencer.torso_frame + 1)
        right_fencer.torso_height = (right_fencer.torso_height*right_fencer.torso_frame + (boxes[i][3] - boxes[i][1]))/(right_fencer.torso_frame + 1)
        right_fencer.torso_frame += 1
    elif classes[i] == 0 and bell_num < 2:
      # Left Fencer
      if boxes[i][0] < frame_width/2:
        # Provides a Running Average
        left_fencer.bellguard_x = (left_fencer.bellguard_x*left_fencer.bell_frame + (boxes[i][0] + boxes[i][2])/2)/(left_fencer.bell_frame + 1)
        left_fencer.bellguard_y = (left_fencer.bellguard_y*left_fencer.bell_frame + (boxes[i][1] + boxes[i][3])/2)/(left_fencer.bell_frame + 1)
        left_fencer.bell_frame += 1
      else:
        right_fencer.bellguard_x = (right_fencer.bellguard_x*right_fencer.bell_frame + (boxes[i][0] + boxes[i][2])/2)/(right_fencer.bell_frame + 1)
        right_fencer.bellguard_y = (right_fencer.bellguard_y*right_fencer.bell_frame + (boxes[i][1] + boxes[i][3])/2)/(right_fencer.bell_frame + 1)
        right_fencer.bell_frame += 1

  return left_fencer, right_fencer